# Text Similarity Scaling for NLP

## Описание
В этом ноутбуке мы будем загружать датасет `deepvk/ru-HNP`, выполнять предобработку текстов и готовить данные для дальнейшего моделирования.

---

## Установка необходимых библиотек

Если у вас еще не установлены некоторые из этих библиотек, вы можете установить их с помощью `pip`. Раскомментируйте и выполните следующую ячейку, если необходимо.


In [1]:
# !pip install pandas numpy nltk pymorphy3 scikit-learn torch matplotlib seaborn datasets tqdm os

In [2]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pymorphy3
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

In [3]:
# Загрузка необходимых ресурсов NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\q\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\q\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\q\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\q\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
ds = load_dataset("deepvk/ru-HNP")

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['query', 'pos', 'neg'],
        num_rows: 100000
    })
    validation: Dataset({
        features: ['query', 'pos', 'neg'],
        num_rows: 590
    })
    test: Dataset({
        features: ['query', 'pos', 'neg'],
        num_rows: 2000
    })
})

In [6]:
# Преобразование тренировочного набора в DataFrame
df_train = pd.DataFrame(ds['train'])

# Преобразование тестового набора в DataFrame
df_test = pd.DataFrame(ds['test'])

# Просмотр первых нескольких строк
df_train.head()


,query,pos,neg
0,С 1952 года после окончания Куйбышевского инду...,[После окончания Куйбышевского индустриального...,[До 1952 года перед окончанием Куйбышевского и...
1,Впадающие реки: Гаронна и Дордонь (образующие ...,[Эстуарий Жиронда формируют впадающие реки: Га...,"[Нет впадающих рек в эстуарий Жиронда, только ..."
2,После смерти отца в 1709 году Людовик унаследо...,"[В 1709 году, после смерти отца, Людовик унасл...",[После смерти отца в 1709 году Людовик лишился...
3,С началом Второй мировой войны клиника перешла...,[После начала Второй мировой войны клиника ста...,[С началом Второй мировой войны клиника перешл...
4,"Родники являются местом паломничества, так как...","[Место паломничества - родники, освященные мит...","[Родники не являются местом паломничества, так..."


In [7]:
# Установка опции для отображения всех строк
pd.set_option('display.max_colwidth', None)  # Показывать полный текст в столбцах

# Просмотр полного содержимого первой строки столбца pos_1
print(df_train['pos'].iloc[0])

['После окончания Куйбышевского индустриального института им. В. В. Куйбышева он начал работать в системе атомной промышленности СССР с 1952 года.', 'С 1952 года после выпуска из Куйбышевского индустриального института им. В. В. Куйбышева он трудился в системе атомной промышленности СССР.', 'После окончания учебы в Куйбышевском индустриальном институте им. В. В. Куйбышева он поступил на работу в систему атомной промышленности СССР в 1952 году.', 'С 1952 года после получения диплома Куйбышевского индустриального института им. В. В. Куйбышева он приступил к работе в системе атомной промышленности СССР.', 'После выпуска из Куйбышевского индустриального института им. В. В. Куйбышева он стал работать в системе атомной промышленности СССР с 1952 года.']


In [8]:
def clean_text(text):
    """
    Очищает текст от специальных символов, оставляя буквы и цифры.
    """
    text = re.sub(r'[^а-яА-ЯёЁ0-9\s]', '', text)
    text = text.lower()
    return text

def tokenize(text):
    """
    Токенизирует текст.
    """
    return nltk.word_tokenize(text)

def lemmatize(tokens):
    """
    Лемматизирует токены с помощью pymorphy3.
    """
    morph = pymorphy3.MorphAnalyzer()
    return [morph.parse(token)[0].normal_form for token in tokens]

def remove_stopwords(tokens):
    """
    Удаляет стоп-слова из токенов.
    """
    stop_words = set(stopwords.words('russian'))
    return [token for token in tokens if token not in stop_words]

def preprocess_text(text):
    """
    Полная предобработка текста.
    """
    text = clean_text(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize(tokens)
    return ' '.join(tokens)


In [9]:
df_train

,query,pos,neg
0,С 1952 года после окончания Куйбышевского индустриального института им. В. В. Куйбышева работал в системе атомной промышленности СССР.,"[После окончания Куйбышевского индустриального института им. В. В. Куйбышева он начал работать в системе атомной промышленности СССР с 1952 года., С 1952 года после выпуска из Куйбышевского индустриального института им. В. В. Куйбышева он трудился в системе атомной промышленности СССР., После окончания учебы в Куйбышевском индустриальном институте им. В. В. Куйбышева он поступил на работу в систему атомной промышленности СССР в 1952 году., С 1952 года после получения диплома Куйбышевского индустриального института им. В. В. Куйбышева он приступил к работе в системе атомной промышленности СССР., После выпуска из Куйбышевского индустриального института им. В. В. Куйбышева он стал работать в системе атомной промышленности СССР с 1952 года.]","[До 1952 года перед окончанием Куйбышевского индустриального института им. В. В. Куйбышева он не работал в системе атомной промышленности СССР., После окончания Куйбышевского индустриального института им. В. В. Куйбышева он не устроился на работу в систему атомной промышленности СССР с 1952 года., До получения диплома Куйбышевского индустриального института им. В. В. Куйбышева он не работал в системе атомной промышленности СССР., После окончания учебы в Куйбышевском индустриальном институте им. В. В. Куйбышева он не приступил к работе в системе атомной промышленности СССР в 1952 году., До выпуска из Куйбышевского индустриального института им. В. В. Куйбышева он не трудился в системе атомной промышленности СССР.]"
1,"Впадающие реки: Гаронна и Дордонь (образующие эстуарий Жиронда), Луара, Адур, Бидасоа.","[Эстуарий Жиронда формируют впадающие реки: Гаронна и Дордонь, а также Луара, Адур и Бидасоа., Гаронна и Дордонь впадают в эстуарий Жиронда вместе с реками Луара, Адур и Бидасоа., Реки Луара, Адур и Бидасоа присоединяются к Гаронне и Дордони, формируя эстуарий Жиронда., Дордонь и Гаронна, а также Луара, Адур и Бидасоа - важные реки, формирующие эстуарий Жиронда., Луара, Адур и Бидасоа впадают в эстуарий Жиронда вместе с Гаронной и Дордонью.]","[Нет впадающих рек в эстуарий Жиронда, только одна большая река., Эстуарий Жиронда формируется только одной рекой, без других впадающих рек., Гаронна и Дордонь не имеют никакого отношения к эстуарию Жиронда, это совершенно разные реки., Луара, Адур и Бидасоа не соединяются с эстуарием Жиронда, они течут в другом направлении., Эстуарий Жиронда образуется только одной рекой, Гаронной, без участия других рек.]"
2,"После смерти отца в 1709 году Людовик унаследовал его титулы, должности и владения.","[В 1709 году, после смерти отца, Людовик унаследовал его титулы, должности и владения., Людовик унаследовал титулы, должности и владения от отца после его смерти в 1709 году., После смерти отца в 1709 году Людовик получил его титулы, должности и владения., Владения, титулы и должности были унаследованы Людовиком после смерти отца в 1709 году., Унаследовав титулы, должности и владения после смерти отца в 1709 году, Людовик стал их обладателем.]","[После смерти отца в 1709 году Людовик лишился его титулов, должностей и владений., Людовик отказался от титулов, должностей и владений своего отца после его смерти в 1709 году., После смерти отца в 1709 году Людовик отрёкся от его титулов, должностей и владений., В 1709 году, после смерти отца, Людовик потерял его титулы, должности и владения., Людовик не наследовал титулы, должности и владения от отца после его смерти в 1709 году.]"
3,"С началом Второй мировой войны клиника перешла в ведение Немецкого института психологических исследований и психотерапии под руководством Маттиаса Генриха Гёринга, где Джон трудился вместе с Вернером Кемпером, занимаясь не только исследованиями, но оказывая помощь евреям и интернированным иностранным рабочим.","[После начала Второй мировой войны клиника стала частью Немецкого института исследований психологии и психотерапии, возглавляемо

In [10]:
df_test

,query,pos,neg
0,Отвечал за работу партии и внес свой вклад в 1936 году в урегулирование инцидента Сиань.,"[Внес свой вклад в урегулирование инцидента Сиань и отвечал за работу партии в 1936 году., В 1936 году отвечал за работу партии и активно участвовал в урегулировании инцидента Сиань., В урегулировании инцидента Сиань в 1936 году принимал участие и отвечал за работу партии., В 1936 году активно участвовал в урегулировании инцидента Сиань и отвечал за работу партии., Ответственно относился к работе партии и внёс свой вклад в урегулирование инцидента Сиань в 1936 году.]","[Не занимался работой партии и не принимал участия в урегулировании инцидента Сиань в 1936 году., Игнорировал работу партии и не вносил никакого вклада в урегулирование инцидента Сиань в 1936 году., Не участвовал в урегулировании инцидента Сиань и не отвечал за работу партии в 1936 году., Не имел никакого вклада в урегулирование инцидента Сиань и не отвечал за работу партии в 1936 году., Был равнодушен к работе партии и не принимал участия в урегулировании инцидента Сиань в 1936 году.]"
1,"Более сложные модели с поддержкой шифрования данных и аутентификацией используются в ситуациях, где важна безопасность, таких как удаленное управление критическими системами, управления механизмами на заводе и промышленных предприятиях, в том числе через Интернет, где безопасность имеет первостепенное значение.","[Более сложные модели, обладающие функциями шифрования и аутентификации, применяются в условиях, где безопасность играет ключевую роль, например, для удаленного управления критическими системами, управления механизмами на заводе и в промышленных предприятиях, включая сеть Интернет, где безопасность находится на первом месте., В ситуациях, где безопасность является приоритетной, используются более сложные модели с возможностью шифрования данных и проверки подлинности, например, для удаленного управления критическими системами, управления механизмами на заводе и в промышленных предприятиях, включая сеть Интернет., Более сложные модели, оснащенные шифрованием данных и аутентификацией, применяются в ситуациях, где безопасность играет важную роль, например, для удаленного управления критическими системами, управления механизмами на заводе и в промышленных предприятиях, включая Интернет, где безопасность имеет высший приоритет., Более сложные модели, которые поддерживают шифрование данных и аутентификацию, используются в ситуациях, где безопасность имеет большое значение, например, для удаленного управления критическими системами, управления механизмами на заводе и в промышленных предприятиях, в том числе через Интернет., Модели с расширенными возможностями шифрования данных и проверки подлинности применяются в ситуациях, где безопасность является ключевым аспектом, таким как удаленное управление критическими системами, управление механизмами на производственных предприятиях и в других сферах, включая использование Интернета, где безопасность играет важную роль.]","[Простые модели без шифрования данных и аутентификации используются в ситуациях, где безопасность не так важна, например, для управления не критическими системами, управления механизмами на заводе и промышленных предприятиях, в том числе через Интернет, где безопасность не на первом плане., Модели без шифрования данных и проверки подлинности используются в ситуациях, где безопасность не стоит на первом месте, например, для удаленного управления не критическими системами, управления механизмами на заводе и в промышленных предприятиях, включая сеть Интернет., Простые модели без поддержки шифрования данных и аутентификации применяются в условиях, где безопасность не является первоочередной задачей, например, для удаленного управления не критическими системами, управления механизмами на заводе и в промышленных предприятиях, в том числе через Интернет., Простые модели, которые не обеспечивают шифрование данных и аутентификацию, используются в ситуациях, где безопасность не столь важна, например, для удаленного уп

In [11]:
# Разделение на новые столбцы
pos_df = pd.DataFrame(df_test['pos'].tolist(), index=df_test.index).add_prefix('pos_')
neg_df = pd.DataFrame(df_test['neg'].tolist(), index=df_test.index).add_prefix('neg_')

# Объединение обратно с оригинальным DataFrame
df_test = pd.concat([df_test.drop(['pos', 'neg'], axis=1), pos_df, neg_df], axis=1)

df_test


,query,pos_0,pos_1,pos_2,pos_3,pos_4,neg_0,neg_1,neg_2,neg_3,neg_4
0,Отвечал за работу партии и внес свой вклад в 1936 году в урегулирование инцидента Сиань.,Внес свой вклад в урегулирование инцидента Сиань и отвечал за работу партии в 1936 году.,В 1936 году отвечал за работу партии и активно участвовал в урегулировании инцидента Сиань.,В урегулировании инцидента Сиань в 1936 году принимал участие и отвечал за работу партии.,В 1936 году активно участвовал в урегулировании инцидента Сиань и отвечал за работу партии.,Ответственно относился к работе партии и внёс свой вклад в урегулирование инцидента Сиань в 1936 году.,Не занимался работой партии и не принимал участия в урегулировании инцидента Сиань в 1936 году.,Игнорировал работу партии и не вносил никакого вклада в урегулирование инцидента Сиань в 1936 году.,Не участвовал в урегулировании инцидента Сиань и не отвечал за работу партии в 1936 году.,Не имел никакого вклада в урегулирование инцидента Сиань и не отвечал за работу партии в 1936 году.,Был равнодушен к работе партии и не принимал участия в урегулировании инцидента Сиань в 1936 году.
1,"Более сложные модели с поддержкой шифрования данных и аутентификацией используются в ситуациях, где важна безопасность, таких как удаленное управление критическими системами, управления механизмами на заводе и промышленных предприятиях, в том числе через Интернет, где безопасность имеет первостепенное значение.","Более сложные модели, обладающие функциями шифрования и аутентификации, применяются в условиях, где безопасность играет ключевую роль, например, для удаленного управления критическими системами, управления механизмами на заводе и в промышленных предприятиях, включая сеть Интернет, где безопасность находится на первом месте.","В ситуациях, где безопасность является приоритетной, используются более сложные модели с возможностью шифрования данных и проверки подлинности, например, для удаленного управления критическими системами, управления механизмами на заводе и в промышленных предприятиях, включая сеть Интернет.","Более сложные модели, оснащенные шифрованием данных и аутентификацией, применяются в ситуациях, где безопасность играет важную роль, например, для удаленного управления критическими системами, управления механизмами на заводе и в промышленных предприятиях, включая Интернет, где безопасность имеет высший приоритет.","Более сложные модели, которые поддерживают шифрование данных и аутентификацию, используются в ситуациях, где безопасность имеет большое значение, например, для удаленного управления критическими системами, управления механизмами на заводе и в промышленных предприятиях, в том числе через Интернет.","Модели с расширенными возможностями шифрования данных и проверки подлинности применяются в ситуациях, где безопасность является ключевым аспектом, таким как удаленное управление критическими системами, управление механизмами на производственных предприятиях и в других сферах, включая использование Интернета, где безопасность играет важную роль.","Простые модели без шифрования данных и аутентификации используются в ситуациях, где безопасность не так важна, например, для управления не критическими системами, управления механизмами на заводе и промышленных предприятиях, в том числе через Интернет, где безопасность не на первом плане.","Модели без шифрования данных и проверки подлинности используются в ситуациях, где безопасность не стоит на первом месте, например, для удаленного управления не критическими системами, управления механизмами на заводе и в промышленных предприятиях, включая сеть Интернет.","Простые модели без поддержки шифрования данных и аутентификации применяются в условиях, где безопасность не является первоочередной задачей, например, для удаленного управления не критическими системами, управления механизмами на заводе и в промышленных предприятиях, в том числе через Интернет.","Простые модели, которые не обеспечивают шифрование данных и аутентификацию, используются в ситуациях, где безопасность не 

In [12]:
# Разделение на новые столбцы
pos_df = pd.DataFrame(df_train['pos'].tolist(), index=df_train.index).add_prefix('pos_')
neg_df = pd.DataFrame(df_train['neg'].tolist(), index=df_train.index).add_prefix('neg_')

# Объединение обратно с оригинальным DataFrame
df_train = pd.concat([df_train.drop(['pos', 'neg'], axis=1), pos_df, neg_df], axis=1)

df_train


,query,pos_0,pos_1,pos_2,pos_3,pos_4,neg_0,neg_1,neg_2,neg_3,neg_4
0,С 1952 года после окончания Куйбышевского индустриального института им. В. В. Куйбышева работал в системе атомной промышленности СССР.,После окончания Куйбышевского индустриального института им. В. В. Куйбышева он начал работать в системе атомной промышленности СССР с 1952 года.,С 1952 года после выпуска из Куйбышевского индустриального института им. В. В. Куйбышева он трудился в системе атомной промышленности СССР.,После окончания учебы в Куйбышевском индустриальном институте им. В. В. Куйбышева он поступил на работу в систему атомной промышленности СССР в 1952 году.,С 1952 года после получения диплома Куйбышевского индустриального института им. В. В. Куйбышева он приступил к работе в системе атомной промышленности СССР.,После выпуска из Куйбышевского индустриального института им. В. В. Куйбышева он стал работать в системе атомной промышленности СССР с 1952 года.,До 1952 года перед окончанием Куйбышевского индустриального института им. В. В. Куйбышева он не работал в системе атомной промышленности СССР.,После окончания Куйбышевского индустриального института им. В. В. Куйбышева он не устроился на работу в систему атомной промышленности СССР с 1952 года.,До получения диплома Куйбышевского индустриального института им. В. В. Куйбышева он не работал в системе атомной промышленности СССР.,После окончания учебы в Куйбышевском индустриальном институте им. В. В. Куйбышева он не приступил к работе в системе атомной промышленности СССР в 1952 году.,До выпуска из Куйбышевского индустриального института им. В. В. Куйбышева он не трудился в системе атомной промышленности СССР.
1,"Впадающие реки: Гаронна и Дордонь (образующие эстуарий Жиронда), Луара, Адур, Бидасоа.","Эстуарий Жиронда формируют впадающие реки: Гаронна и Дордонь, а также Луара, Адур и Бидасоа.","Гаронна и Дордонь впадают в эстуарий Жиронда вместе с реками Луара, Адур и Бидасоа.","Реки Луара, Адур и Бидасоа присоединяются к Гаронне и Дордони, формируя эстуарий Жиронда.","Дордонь и Гаронна, а также Луара, Адур и Бидасоа - важные реки, формирующие эстуарий Жиронда.","Луара, Адур и Бидасоа впадают в эстуарий Жиронда вместе с Гаронной и Дордонью.","Нет впадающих рек в эстуарий Жиронда, только одна большая река.","Эстуарий Жиронда формируется только одной рекой, без других впадающих рек.","Гаронна и Дордонь не имеют никакого отношения к эстуарию Жиронда, это совершенно разные реки.","Луара, Адур и Бидасоа не соединяются с эстуарием Жиронда, они течут в другом направлении.","Эстуарий Жиронда образуется только одной рекой, Гаронной, без участия других рек."
2,"После смерти отца в 1709 году Людовик унаследовал его титулы, должности и владения.","В 1709 году, после смерти отца, Людовик унаследовал его титулы, должности и владения.","Людовик унаследовал титулы, должности и владения от отца после его смерти в 1709 году.","После смерти отца в 1709 году Людовик получил его титулы, должности и владения.","Владения, титулы и должности были унаследованы Людовиком после смерти отца в 1709 году.","Унаследовав титулы, должности и владения после смерти отца в 1709 году, Людовик стал их обладателем.","После смерти отца в 1709 году Людовик лишился его титулов, должностей и владений.","Людовик отказался от титулов, должностей и владений своего отца после его смерти в 1709 году.","После смерти отца в 1709 году Людовик отрёкся от его титулов, должностей и владений.","В 1709 году, после смерти отца, Людовик потерял его титулы, должности и владения.","Людовик не наследовал титулы, должности и владения от отца после его смерти в 1709 году."
3,"С началом Второй мировой войны клиника перешла в ведение Немецкого института психологических исследований и психотерапии под руководством Маттиаса Генриха Гёринга, где Джон трудился вместе с Вернером Кемпером, занимаясь не только исследованиями, но оказывая помощь евреям и интернированным иностранным рабочим.","После начала Второй мировой войны клиника стала частью Немецкого института исследова

In [13]:
# Выбор 10% от df_train
df_train = df_train.sample(frac=0.1, random_state=42).reset_index(drop=True)
df_train

,query,pos_0,pos_1,pos_2,pos_3,pos_4,neg_0,neg_1,neg_2,neg_3,neg_4
0,Клини́ческий переулок — название улиц в различных населённых пунктах государств бывшего СССР.,Переулок с названием Клини́ческий расположен в разных населённых пунктах государств бывшего СССР.,В различных городах бывшего СССР есть улицы с названием Клини́ческий переулок.,На улице Клини́ческий переулок можно встретить в разных населённых пунктах бывшего СССР.,Улицы с названием Клини́ческий переулок пролегают через различные населённые пункты государств бывшего СССР.,Название Клини́ческий переулок используется для улиц в разных населённых пунктах бывшего СССР.,Бывший СССР не имеет улиц с названием Клини́ческий переулок.,"Клини́ческий переулок — улица, которой нет в различных населённых пунктах бывшего СССР.",В городах бывшего СССР не существует улицы с названием Клини́ческий переулок.,Название Клини́ческий переулок не используется для улиц в государствах бывшего СССР.,Улицы в различных населённых пунктах бывшего СССР не называются Клини́ческий переулок.
1,Также см. схему «Развитие сети городского транспорта» — из Генерального плана Новосибирска до 2030 года: ).,Изучите также план развития городского транспорта Новосибирска до 2030 года.,Посмотрите схему развития городского транспорта в Новосибирске до 2030 года.,Приглядитесь к плану развития городского транспорта Новосибирска до 2030 года.,Рассмотрите также план развития городского транспорта Новосибирска на 2030 год.,Взгляните на схему развития городского транспорта в Новосибирске на период до 2030 года.,Не обращайте внимания на схему «Развитие сети городского транспорта» в Генеральном плане Новосибирска до 2030 года.,Игнорируйте план развития городского транспорта Новосибирска до 2030 года.,Не следуйте схеме развития городского транспорта в Новосибирске до 2030 года.,Не учитывайте план развития городского транспорта Новосибирска на 2030 год.,Не принимайте во внимание схему развития городского транспорта в Новосибирске на период до 2030 года.
2,После окончания Московского среднего технического училища (1918) работал электромехаником в Московском практическом электротехническом институте и одновременно учился там же (окончил в 1921 г. со званием инженера по электрооборудованию фабрик и заводов).,"Работая электромехаником в Московском практическом электротехническом институте, после окончания Московского среднего технического училища (1918), он одновременно учился там же и успешно закончил в 1921 году, получив звание инженера по электрооборудованию фабрик и заводов.","После окончания Московского среднего технического училища (1918), он работал электромехаником в Московском практическом электротехническом институте, а также учился в этом же институте, где закончил обучение в 1921 году с званием инженера по электрооборудованию фабрик и заводов.","Электромехаником в Московском практическом электротехническом институте он работал после окончания Московского среднего технического училища в 1918 году, параллельно учась в этом же институте и успешно завершив обучение в 1921 году, став инженером по электрооборудованию фабрик и заводов.","После окончания Московского среднего технического училища в 1918 году, он приступил к работе электромехаником в Московском практическом электротехническом институте, где параллельно проходил обучение и в 1921 году успешно стал инженером по электрооборудованию фабрик и заводов.","Электромехаником в Московском практическом электротехническом институте, после окончания Московского среднего технического училища в 1918 году, он продолжил обучение в том же институте и завершил его в 1921 году, получив звание инженера по электрооборудованию фабрик и заводов.","Не окончив Московское среднее техническое училище в 1918 году, он не стал электромехаником в Московском практическом электротехническом институте и не учился там же, не закончив обучение в 1921 году со званием инженера по электрооборудованию фабрик и заводов.","После окончания Московского среднего технического училища в 1918 году, он не р

In [14]:
# Применение предобработки к столбцам
for col in tqdm(df_train.columns):
    df_train[col] = df_train[col].apply(preprocess_text)

for col in tqdm(df_test.columns):
    df_test[col] = df_test[col].apply(preprocess_text)



100%|█████████████████████████████████████████████████████████████████████████████████| 11/11 [30:20<00:00, 165.53s/it]


In [15]:
# Просмотр предобработанных данных
df_train.head()

,query,pos_0,pos_1,pos_2,pos_3,pos_4,neg_0,neg_1,neg_2,neg_3,neg_4
0,клинический переулок название улица различный населить пункт государство бывший ссср,переулок название клинический расположить разный населить пункт государство бывший ссср,различный город бывший ссср улица название клинический переулок,улица клинический переулок встретить разный населить пункт бывший ссср,улица название клинический переулок пролегать различный населённый пункт государство бывший ссср,название клинический переулок использоваться улица разный населить пункт бывший ссср,бывший ссср иметь улица название клинический переулок,клинический переулок улица который различный населить пункт бывший ссср,город бывший ссср существовать улица название клинический переулок,название клинический переулок использоваться улица государство бывший ссср,улица различный населить пункт бывший ссср называться клинический переулок
1,также смотреть схема развитие сеть городской транспорт генеральный план новосибирск 2030 год,изучить также план развитие городской транспорт новосибирск 2030 год,посмотреть схема развитие городской транспорт новосибирск 2030 год,приглядеться план развитие городской транспорт новосибирск 2030 год,рассмотреть также план развитие городской транспорт новосибирск 2030 год,взглянуть схема развитие городской транспорт новосибирск период 2030 год,обращать внимание схема развитие сеть городской транспорт генеральный план новосибирск 2030 год,игнорировать план развитие городской транспорт новосибирск 2030 год,следовать схема развитие городской транспорт новосибирск 2030 год,учитывать план развитие городской транспорт новосибирск 2030 год,принимать внимание схема развитие городской транспорт новосибирск период 2030 год
2,окончание московский среднее технический училище 1918 работать электромеханик московский практический электротехнический институт одновременно учиться окончить 1921 г звание инженер электрооборудование фабрика завод,работать электромеханик московский практический электротехнический институт окончание московский среднее технический училище 1918 одновременно учиться успешно закончить 1921 год получить звание инженер электрооборудование фабрика завод,окончание московский среднее технический училище 1918 работать электромеханик московский практический электротехнический институт также учиться институт закончить обучение 1921 год звание инженер электрооборудование фабрика завод,электромеханик московский практический электротехнический институт работать окончание московский среднее технический училище 1918 год параллельно учиться институт успешно завершить обучение 1921 год стать инженер электрооборудование фабрика завод,окончание московский среднее технический училище 1918 год приступить работа электромеханик московский практический электротехнический институт параллельно проходить обучение 1921 год успешно стать инженер электрооборудование фабрика завод,электромеханик московский практический электротехнический институт окончание московский среднее технический училище 1918 год продолжить обучение институт завершить 1921 год получить звание инженер электрооборудование фабрика завод,окончить московский средний технический училище 1918 год стать электромеханик московский практический электротехнический институт учиться закончить обучение 1921 год звание инженер электрооборудование фабрика завод,окончание московский среднее технический училище 1918 год работать электромеханик московский практический электротехнический институт учиться получить звание инженер электрооборудование фабрика завод 1921 год,окончание московский среднее технический училище 1918 год работать электромеханик московский практический электротехнический институт учиться получить 1921 год звание инженер электрооборудование фабрика завод,работать электромеханик московский практический электротехнический институт окончание московский среднее технический училище 1918 год учиться закончить обучение 1921 год звание инженер электрооборудование фабрика завод,окончание московский ср

In [16]:
# Просмотр предобработанных данных
df_test.head()

,query,pos_0,pos_1,pos_2,pos_3,pos_4,neg_0,neg_1,neg_2,neg_3,neg_4
0,отвечать работа партия внести свой вклад 1936 год урегулирование инцидент сианя,внести свой вклад урегулирование инцидент сианя отвечать работа партия 1936 год,1936 год отвечать работа партия активно участвовать урегулирование инцидент сианя,урегулирование инцидент сианя 1936 год принимать участие отвечать работа партия,1936 год активно участвовать урегулирование инцидент сианя отвечать работа партия,ответственно относиться работа партия внести свой вклад урегулирование инцидент сианя 1936 год,заниматься работа партия принимать участие урегулирование инцидент сианя 1936 год,игнорировать работа партия вносить никакой вклад урегулирование инцидент сианя 1936 год,участвовать урегулирование инцидент сианя отвечать работа партия 1936 год,иметь никакой вклад урегулирование инцидент сианя отвечать работа партия 1936 год,равнодушный работа партия принимать участие урегулирование инцидент сианя 1936 год
1,сложный модель поддержка шифрование данные аутентификация использоваться ситуация важный безопасность такой удалённый управление критический система управление механизм завод промышленный предприятие число интернет безопасность иметь первостепенный значение,сложный модель обладать функция шифрование аутентификация применяться условие безопасность играть ключевой роль например удалённый управление критический система управление механизм завод промышленный предприятие включая сеть интернет безопасность находиться первый место,ситуация безопасность являться приоритетный использоваться сложный модель возможность шифрование данные проверка подлинность например удалённый управление критический система управление механизм завод промышленный предприятие включая сеть интернет,сложный модель оснастить шифрование данные аутентификация применяться ситуация безопасность играть важный роль например удалённый управление критический система управление механизм завод промышленный предприятие включая интернет безопасность иметь высокий приоритет,сложный модель который поддерживать шифрование данные аутентификация использоваться ситуация безопасность иметь большой значение например удалённый управление критический система управление механизм завод промышленный предприятие число интернет,модель расширить возможность шифрование данные проверка подлинность применяться ситуация безопасность являться ключевой аспект такой удалённый управление критический система управление механизм производственный предприятие другой сфера включая использование интернет безопасность играть важный роль,простой модель шифрование данные аутентификация использоваться ситуация безопасность важный например управление критический система управление механизм завод промышленный предприятие число интернет безопасность первый план,модель шифрование данные проверка подлинность использоваться ситуация безопасность стоить первый место например удалённый управление критический система управление механизм завод промышленный предприятие включая сеть интернет,простой модель поддержка шифрование данные аутентификация применяться условие безопасность являться первоочередной задача например удалённый управление критический система управление механизм завод промышленный предприятие число интернет,простой модель который обеспечивать шифрование данные аутентификация использоваться ситуация безопасность столь важный например удалённый управление критический система управление механизм завод промышленный предприятие включая интернет,модель сложный защита данные аутентификация применяться условие безопасность иметь высокий приоритет например управление критический система управление механизм завод другой отрасль включая использование интернет безопасность играть ключевой роль
2,это время отплыть два лодка лодка сам генерал уиллер второй получить пробоина ватерлиния ядро пустить берег,два лодка отправиться плавание лодка генерал уиллер другой который получить пробоина ядро выпустить берег,отплыть два лодка лодка генерал уиллер один ко

In [17]:
df_train.to_csv('../data/df_train_10percent.csv', index=False)
df_test.to_csv('../data/df_test.csv', index=False)